# Mount google drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# Download and install requirements

In [ ]:
!cp -r gdrive/MyDrive/NLP/image_search_data ./

In [ ]:
!pip install transformers
!pip install -q kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 99.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 103.1 MB/s eta 0:00:00


In [ ]:
# you should clone git repository (https://github.com/NLP-Final-Projects/image-search/tree/base-model)

In [ ]:
import pandas as pd
import os
from os import listdir
from os.path import isfile, join
from src.train.training import train_clip
import yaml
import torch
from transformers import TrainingArguments, AutoTokenizer, CLIPFeatureExtractor
from transformers import CLIPVisionModel, AutoModel
from sklearn.model_selection import train_test_split
from src.train.dataset import CLIPDataset
from src.train.model import get_clip_model
from src.train.trainer import CLIPTrainer
from src.train.utils import get_num_processors
from src.train.utils import clear_gpu

In [ ]:
!nvidia-smi

Wed Mar  8 11:29:44 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   74C    P0    32W /  70W |   7225MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Load and process dataset

In [ ]:
df = pd.read_csv('image_search_data/dataset.csv')
onlyfiles = [f for f in listdir('image_search_data/images/') if isfile(join('image_search_data/images/', f))]
df = df[df.image.apply(lambda x: os.path.basename(x)).isin(onlyfiles)]
df.image = df.image.apply(lambda x: 'image_search_data/images/'+os.path.basename(x))
df.dropna().to_csv('dataset.csv', index=False)

In [ ]:
with open("src/train/params.yaml", "r") as stream:
    params = yaml.safe_load(stream)

train_params = params['train']
train_params

{'dataset_path': 'dataset.csv',
 'test_size': 0.2,
 'text_model': 'm3hrdadfi/roberta-zwnj-wnli-mean-tokens',
 'image_model': 'openai/clip-vit-base-patch32',
 'batch_size': 64,
 'image_size': 224,
 'max_len': 80,
 'images_folder_path': 'image_search_data/images',
 'clip_config': {'num_hidden_layers': 0,
  'max_position_embeddings': 0,
  'vocab_size': 0,
  'hidden_size': 1,
  'patch_size': 1}}

# Train Model

In [ ]:
train_clip(**train_params)

Some weights of the model checkpoint at openai/clip-vit-base-patch32 were not used when initializing CLIPVisionModel: ['text_model.encoder.layers.3.self_attn.out_proj.weight', 'text_model.encoder.layers.2.self_attn.out_proj.bias', 'text_model.encoder.layers.2.mlp.fc1.weight', 'text_model.encoder.layers.7.mlp.fc2.weight', 'text_model.encoder.layers.3.self_attn.q_proj.bias', 'text_model.final_layer_norm.weight', 'text_model.encoder.layers.10.self_attn.out_proj.weight', 'text_model.encoder.layers.4.mlp.fc1.weight', 'text_model.encoder.layers.10.mlp.fc2.bias', 'text_model.encoder.layers.4.self_attn.v_proj.bias', 'text_model.encoder.layers.3.self_attn.out_proj.bias', 'text_model.encoder.layers.4.self_attn.q_proj.bias', 'text_model.encoder.layers.10.mlp.fc1.weight', 'text_model.encoder.layers.0.self_attn.out_proj.weight', 'text_model.encoder.layers.1.self_attn.out_proj.bias', 'text_model.encoder.layers.6.layer_norm2.bias', 'text_model.encoder.layers.11.mlp.fc1.weight', 'text_model.encoder.la

start training


Step,Training Loss,Validation Loss
100,2.383100,2.274366
200,1.800300,1.803008
300,1.599700,1.487533
400,1.363600,1.330889
500,1.210900,1.176592
600,1.160700,1.139223
700,1.167800,1.048557
800,1.135500,0.991372
900,0.994100,0.924928


***** Running Evaluation *****
  Num examples = 1620
  Batch size = 32
***** Running Evaluation *****
  Num examples = 1620
  Batch size = 32
***** Running Evaluation *****
  Num examples = 1620
  Batch size = 32
***** Running Evaluation *****
  Num examples = 1620
  Batch size = 32
***** Running Evaluation *****
  Num examples = 1620
  Batch size = 32
Saving model checkpoint to clip-fa/checkpoint-500
Configuration saved in clip-fa/checkpoint-500/config.json
Model weights saved in clip-fa/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1620
  Batch size = 32
***** Running Evaluation *****
  Num examples = 1620
  Batch size = 32
***** Running Evaluation *****
  Num examples = 1620
  Batch size = 32
***** Running Evaluation *****
  Num examples = 1620
  Batch size = 32


Training completed. Do not forget to share your model on huggingface.co/models =)


Configuration saved in image-fa-search-text/config.json


saving fine-tuned model


Model weights saved in image-fa-search-text/pytorch_model.bin
tokenizer config file saved in image-fa-search-text/tokenizer_config.json
Special tokens file saved in image-fa-search-text/special_tokens_map.json
Configuration saved in image-fa-search-vision/config.json
Model weights saved in image-fa-search-vision/pytorch_model.bin


# Save fine-tuned model to drive

In [ ]:
!cp -r ./image-fa-search-text/ gdrive/MyDrive/NLP/image-search

In [ ]:
!cp -r ./image-fa-search-vision/ gdrive/MyDrive/NLP/image-search